In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
from scipy.stats import multivariate_normal

# Add the parent directory of simpleparameterisation to the sys.path list
sys.path.append('../')

# Now you can import simpleparameterisation from the new location
import TPCevt as TPC

det = TPC.Detector("Simple TPC with Atmospheric Ar (default values)")


In [2]:
# Initial Parameter
pitch_x = 0.1
pitch_y = 0.1
pitch_z = 0.1
std = det.PSFstd = 1
thresh = 1e-20
grid_size = 500
rv = multivariate_normal([0,0,0], np.diag([std, std, std]))

In [3]:
def gainAndReadout(DriftedEvt, det, nsigma_extend=3, thresh=1e1):
    minvals = np.array([DriftedEvt[1].min(),DriftedEvt[2].min(), DriftedEvt[3].min() ])
    maxvals = np.array([DriftedEvt[1].max(),DriftedEvt[2].max(), DriftedEvt[3].max() ])
    nsigma_extend = 3
    PSFmean_extend = nsigma_extend * det.PSFstd + det.PSFmean
    x_range = (minvals[0] - PSFmean_extend, maxvals[0] + PSFmean_extend)
    y_range = (minvals[1] - PSFmean_extend, maxvals[1] + PSFmean_extend)
    dt_range = (minvals[2] - nsigma_extend * det.gain_sigma_t, maxvals[2] + nsigma_extend * det.gain_sigma_t)
    # Calculate ReadoutGrid
    ReadoutGrid = np.mgrid[
                np.floor(x_range[0] / det.pitch_x) * det.pitch_x:
                np.ceil(x_range[1] / det.pitch_x) * det.pitch_x:
                det.pitch_x,
                np.floor(y_range[0] / det.pitch_y) * det.pitch_y:
                np.ceil(y_range[1] / det.pitch_y) * det.pitch_y:
                det.pitch_y,
                np.floor(dt_range[0] / det.samplerate) * det.samplerate:
                np.ceil(dt_range[1] / det.samplerate) * det.samplerate:
                det.samplerate
                ]

    pos = np.stack(ReadoutGrid, axis=3)
    ReadoutEvt = None
    for i in range(len(DriftedEvt[1])):
        thisGain = np.random.exponential(scale=det.gain_mean)
        rv = multivariate_normal([DriftedEvt[1][i], DriftedEvt[2][i], DriftedEvt[3][i]], np.diag([det.PSFstd, det.PSFstd, det.gain_sigma_t]))
        if ReadoutEvt is None:
            #The factor pitch_x*pitch_y*sample_rate converts from probability density 
            #to 'normalised' probability
            ReadoutEvt = rv.pdf(pos)*det.pitch_x*det.pitch_y*det.samplerate*thisGain
        else:
            ReadoutEvt += rv.pdf(pos)*det.pitch_x*det.pitch_y*det.samplerate*thisGain
    themask = ReadoutEvt >= thresh
    if np.sum(themask) == 0:
        print('No channels above threshold!')
        return pd.DataFrame(columns=['x', 'y', 'dt', 'Nel'])
    try:
        return pd.DataFrame({'x': ReadoutGrid[0][themask].flatten(), 'y': ReadoutGrid[1][themask].flatten(), 
            'dt': ReadoutGrid[2][themask].flatten(), 'Nel': ReadoutEvt[themask]})
    except:
        print(f'DriftedEvt: {DriftedEvt}')
        print(f'themask: {themask}\nReadoutGrid[0]: {ReadoutGrid[0]}\nReadoutEvt: {ReadoutEvt}\nReadoutGrid: {ReadoutGrid}')
        print(f'ReadoutGrid.shape: {ReadoutGrid.shape}, ReadoutEvt.shape: {ReadoutEvt.shape}, themask.shape = {themask.shape}')
        raise